In [26]:
import torch
import math
import networkx as nx
from torch_geometric.data import Data
from torch_geometric.utils import convert
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree


In [27]:
edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous())

In [28]:
print(data)

Data(edge_index=[2, 4], x=[3, 1])


In [29]:
ls

GraphUB.ipynb                       Vaga8Nov-link-list_out.graphml
Marta_Rovira-link-list_out.graphml  nochebuena-link-list_out.graphml


In [5]:
G = nx.read_graphml("Marta_Rovira-link-list_out.graphml")

In [16]:
G.nodes

NodeView(('pepimares', 'jonathanmartinz', 'SylviLG', 'VilaWeb', 'martarovira', 'protestona1', 'LandaTxema', 'dihanaris', 'JohnNieve', '87JF87', 'Albert_Rivera', 'JULIAPACOIRENE', 'BobEstropajo', 'tintintalca', 'AlfonsoRojoPD', 'benaiges78', 'soler_toni', 'annamariasastre', 'antoniobanos_', 'vngsergi', 'Josesegoviapa', 'JuanGosset', 'ferranrmdeusbcn', 'ferranq9', 'HumphreyDiuQue', 'Rafa10894276', 'lluis_llach', 'montsemaspi', 'MardraggomezMar', 'Tonicanto1', 'jordi_canyas', 'JosPastr', 'xmarchmarti', 'vpartal', 'biolucho2005', 'esrafiki', 'XSalaimartin', 'jordimarcar', 'eldiarioes', 'jesusmarana', 'MomentsES', 'oscarcerezal', 'indpcom', 'DeOrcajo', 'ImmaPuig', 'Lo_Tio_Collons', 'Gatosombra1', 'arnaucerda', 'Europeas_14', 'gonnassau', 'RevoluSSSion', 'Carloscatetolan', 'Enric_Hernandez', 'gonzalezllobet', 'CatalunyaRadio', '324cat', 'Cati_Politica', 'IMendezdeVigo', 'Whit3P0wder', 'oriolguellipuig', 'Lasarico23', 'gerardnus', 'rac1', 'Eli_F_Ros', 'MMSLibertad', 'Paco_Glez_', 'CiudadanosC

In [6]:
nx.info(G)

'Name: \nType: DiGraph\nNumber of nodes: 26832\nNumber of edges: 53285\nAverage in degree:   1.9859\nAverage out degree:   1.9859'

In [7]:
print(G.nodes["ElisendaIngles"])

{'participation index': 5.55112e-16, 'dispersion index': 0.0, 'global hubness': -1.49126, 'local hubness': -1.0999, 'community': 1}


In [9]:
p_i = nx.get_node_attributes(G,'participation index')
not_nan = [honk for honk in p_i.keys() if (math.isnan(p_i[honk]) == False)]

In [10]:
d_i = nx.get_node_attributes(G,'dispersion index')
di_not_0 = [honk for honk in d_i.keys() if d_i[honk] > 0]
print(di_not_0)

['SylviLG', 'LandaTxema', 'annamariasastre', 'ferranq9', 'MardraggomezMar', 'jordimarcar', 'indpcom', 'ImmaPuig', 'Carloscatetolan', 'gonzalezllobet', 'CatalunyaRadio', 'Cati_Politica', 'gerardnus', 'ignatiu_reilly', 'elviradesubirat', 'JuntsPelSi_Cat', 'MoneyCorruption', 'SoZePequenho', 'ESTHEREGEA3', 'eloresotzet', 'LAROEDU', 'clarcadi', 'Davidaf55', 'elsomatent', 'asiandafrica', 'egel71', 'oscarguallar', 'jogevis', 'Montseblanca', 'josmato', 'joancguerrero', 'RogerC88', 'catala1962', 'MnicaRebullCome', 'agora0813', 'AmorosaVenus', 'litxuetxarri', 'DrusoJGT', 'SI_democraciaSI', 'NottobeTobeor', 'Sin_ve_llo', 'RosaSancho8', 'MdelMarRV', 'JredondoK', 'GquirogaGonzalo', 'betomorrowdawn', 'manel_martinezg', 'itsjuliaolive', 'FraJeremies', 'rakiblanes', 'pumuky68', 'noecatnord', 'alexeina', 'carthografo', 'antoniaserrano2', 'guillermo_663', 'bac_just', 'RosaFregi', 'Almumellaman', 'Barbara_pla', 'amelasadko', 'joeabenpresa', 'mariossans', 'Marga66304808', 'xavier__perez', 'Gemmacdev', 'Je

In [11]:
nx.get_node_attributes(G,'global hubness')

{'pepimares': -1.49126,
 'jonathanmartinz': -1.99307,
 'SylviLG': 0.0141761,
 'VilaWeb': -0.989448,
 'martarovira': -1.49126,
 'protestona1': -1.99307,
 'LandaTxema': -0.989448,
 'dihanaris': -1.49126,
 'JohnNieve': -1.99307,
 '87JF87': -1.49126,
 'Albert_Rivera': -1.99307,
 'JULIAPACOIRENE': -1.49126,
 'BobEstropajo': -1.99307,
 'tintintalca': -1.49126,
 'AlfonsoRojoPD': -1.99307,
 'benaiges78': -1.49126,
 'soler_toni': -1.49126,
 'annamariasastre': 0.0141761,
 'antoniobanos_': -1.99307,
 'vngsergi': -1.49126,
 'Josesegoviapa': -1.49126,
 'JuanGosset': -0.989448,
 'ferranrmdeusbcn': -1.49126,
 'ferranq9': -0.989448,
 'HumphreyDiuQue': 0.0141761,
 'Rafa10894276': -1.49126,
 'lluis_llach': -0.487636,
 'montsemaspi': -0.989448,
 'MardraggomezMar': 0.515988,
 'Tonicanto1': -1.99307,
 'jordi_canyas': -1.49126,
 'JosPastr': -1.99307,
 'xmarchmarti': -0.487636,
 'vpartal': -1.99307,
 'biolucho2005': -1.49126,
 'esrafiki': -1.49126,
 'XSalaimartin': -1.49126,
 'jordimarcar': 1.51961,
 'eldiar

In [12]:
nx.get_node_attributes(G,'local hubness')

{'pepimares': -1.0999,
 'jonathanmartinz': -1.69133,
 'SylviLG': 0.506042,
 'VilaWeb': -0.508465,
 'martarovira': -1.0999,
 'protestona1': -1.69133,
 'LandaTxema': -0.628809,
 'dihanaris': -1.41328,
 'JohnNieve': -1.93126,
 '87JF87': -1.41328,
 'Albert_Rivera': -1.93126,
 'JULIAPACOIRENE': -0.867809,
 'BobEstropajo': -1.52524,
 'tintintalca': -1.41328,
 'AlfonsoRojoPD': -1.93126,
 'benaiges78': -1.0999,
 'soler_toni': -1.0999,
 'annamariasastre': 0.506042,
 'antoniobanos_': -1.69133,
 'vngsergi': -1.0999,
 'Josesegoviapa': -1.41328,
 'JuanGosset': -0.895304,
 'ferranrmdeusbcn': -1.0999,
 'ferranq9': -0.628809,
 'HumphreyDiuQue': 0.506042,
 'Rafa10894276': -1.0999,
 'lluis_llach': 0.0829668,
 'montsemaspi': -0.508465,
 'MardraggomezMar': 0.658623,
 'Tonicanto1': -1.93126,
 'jordi_canyas': -1.41328,
 'JosPastr': -1.93126,
 'xmarchmarti': 0.0829668,
 'vpartal': -1.69133,
 'biolucho2005': -1.41328,
 'esrafiki': -1.0999,
 'XSalaimartin': -1.0999,
 'jordimarcar': 2.20832,
 'eldiarioes': -1.6

In [13]:
nx.get_node_attributes(G,'community')

{'pepimares': 1,
 'jonathanmartinz': 1,
 'SylviLG': 2,
 'VilaWeb': 1,
 'martarovira': 1,
 'protestona1': 1,
 'LandaTxema': 2,
 'dihanaris': 3,
 'JohnNieve': 3,
 '87JF87': 3,
 'Albert_Rivera': 3,
 'JULIAPACOIRENE': 4,
 'BobEstropajo': 4,
 'tintintalca': 3,
 'AlfonsoRojoPD': 3,
 'benaiges78': 1,
 'soler_toni': 1,
 'annamariasastre': 2,
 'antoniobanos_': 1,
 'vngsergi': 1,
 'Josesegoviapa': 3,
 'JuanGosset': 3,
 'ferranrmdeusbcn': 1,
 'ferranq9': 2,
 'HumphreyDiuQue': 2,
 'Rafa10894276': 1,
 'lluis_llach': 1,
 'montsemaspi': 1,
 'MardraggomezMar': 3,
 'Tonicanto1': 3,
 'jordi_canyas': 3,
 'JosPastr': 3,
 'xmarchmarti': 1,
 'vpartal': 1,
 'biolucho2005': 3,
 'esrafiki': 1,
 'XSalaimartin': 1,
 'jordimarcar': 2,
 'eldiarioes': 1,
 'jesusmarana': 3,
 'MomentsES': 2,
 'oscarcerezal': 3,
 'indpcom': 3,
 'DeOrcajo': 3,
 'ImmaPuig': 2,
 'Lo_Tio_Collons': 2,
 'Gatosombra1': 1,
 'arnaucerda': 1,
 'Europeas_14': 3,
 'gonnassau': 3,
 'RevoluSSSion': 3,
 'Carloscatetolan': 3,
 'Enric_Hernandez': 3,
 

In [14]:
print(not_nan)

['pepimares', 'jonathanmartinz', 'SylviLG', 'VilaWeb', 'martarovira', 'protestona1', 'LandaTxema', 'dihanaris', 'JohnNieve', '87JF87', 'Albert_Rivera', 'JULIAPACOIRENE', 'BobEstropajo', 'tintintalca', 'AlfonsoRojoPD', 'benaiges78', 'soler_toni', 'annamariasastre', 'antoniobanos_', 'vngsergi', 'Josesegoviapa', 'JuanGosset', 'ferranrmdeusbcn', 'ferranq9', 'HumphreyDiuQue', 'Rafa10894276', 'lluis_llach', 'montsemaspi', 'MardraggomezMar', 'Tonicanto1', 'jordi_canyas', 'JosPastr', 'xmarchmarti', 'vpartal', 'biolucho2005', 'esrafiki', 'XSalaimartin', 'jordimarcar', 'eldiarioes', 'jesusmarana', 'MomentsES', 'oscarcerezal', 'indpcom', 'DeOrcajo', 'ImmaPuig', 'Lo_Tio_Collons', 'Gatosombra1', 'arnaucerda', 'Europeas_14', 'gonnassau', 'RevoluSSSion', 'Carloscatetolan', 'Enric_Hernandez', 'gonzalezllobet', 'CatalunyaRadio', '324cat', 'Cati_Politica', 'IMendezdeVigo', 'Whit3P0wder', 'oriolguellipuig', 'Lasarico23', 'gerardnus', 'rac1', 'Eli_F_Ros', 'MMSLibertad', 'Paco_Glez_', 'CiudadanosCs', 'PPop

In [24]:
data_marta_rovira = convert.from_networkx(G)

In [25]:
data_marta_rovira

Data(community=[26832], dispersion index=[26832], edge_index=[2, 53285], global hubness=[26832], local hubness=[26832], participation index=[26832])